In [7]:
from astropy.table import Table, join
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import pandas as pd
import statistics as st


ForGroup = Table.read('../../GAMA_DATA/G3CFoFGroupv10.fits')
print(f"ForGroup : {len(ForGroup)}")
## Clean Data

GroupGal = Table.read('../../GAMA_DATA/G3CGalv10.fits')
# Clean data
GroupGal = GroupGal[GroupGal['GroupID'] > 0 ]
print(f"GroupGal : {len(GroupGal)}")

StellarMasses = Table.read('../../GAMA_DATA/StellarMassesv19.fits')
# Clean Data
#StellarMasses = StellarMasses[StellarMasses['uminusr'] > 0.001]
#StellarMasses = StellarMasses[StellarMasses['logmstar'] > 0.001]
#StellarMasses = StellarMasses[StellarMasses['metal'] > 0.001]


StellarMasses = StellarMasses[StellarMasses['uminusr'] > 0.01]
StellarMasses = StellarMasses[StellarMasses['logmstar'] > 0.01]
StellarMasses = StellarMasses[StellarMasses['metal'] > 0.01]
print(f"StellarMasses : {len(StellarMasses)}")


envMeasures = Table.read('../../GAMA_DATA/EnvironmentMeasuresv05.fits')
print(f"EnvMeasures : {len(envMeasures)}")
# Clean Data
envMeasures = envMeasures[envMeasures['SurfaceDensity'] < 50]
envMeasures = envMeasures[envMeasures['AGEDenParFlag'] == 0]
envMeasures = envMeasures[envMeasures['CountInCylFlag'] == 0]
envMeasures = envMeasures[envMeasures['CountInCyl'] > 0]

envClass = Table.read('../../GAMA_DATA/GalaxiesClassifiedv01.fits')
# Clean Data
visualMorph = Table.read('../../GAMA_Data/VisualMorphologyv03.fits')
print(f"visualMorph : {len(visualMorph)}")
## Clean Data
visualMorphElliptical = visualMorph[visualMorph['ELLIPTICAL_CODE'] == 1]
visualMorphSpiral = visualMorph[visualMorph['ELLIPTICAL_CODE'] == 10]

DataLocalGroup = join(GroupGal,envMeasures,keys='CATAID',join_type='inner')
print(f"Join GroupGal & EnvMeasures {len(DataLocalGroup)}")
print('GroupGal & EnvMeasures')
print(DataLocalGroup.colnames)
print(len(DataLocalGroup))

DLG_Groups = DataLocalGroup.group_by('GroupID')
GroupData = DLG_Groups['GroupID','CountInCyl','DistanceTo5nn','SurfaceDensity','AGEDenPar'].groups.aggregate(np.sum)
dfe = GroupData.to_pandas()

dfDLG = DLG_Groups.to_pandas()
dfc = dfDLG.groupby(['GroupID']).size().reset_index(name='Counts')

dfG = ForGroup.to_pandas()
#print(dfG.columns)
df1 = pd.merge(dfe, dfc, on=["GroupID", "GroupID"])
df  = pd.merge(df1,dfG,on=["GroupID", "GroupID"])
print(df.columns)

from sklearn import linear_model
#import statsmodels.api as sm

Y = df['TotRmag']

for m in ['CountInCyl','DistanceTo5nn','SurfaceDensity','AGEDenPar'] :
    X = df[[m,'Counts']] 
    # with sklearn
    regr = linear_model.LinearRegression()
    r = regr.fit(X, Y)
    print(f'{m} Coefficients: {regr.coef_} Intercept: {regr.intercept_}')
    print(dir(regr))
   


    

ForGroup : 26194
GroupGal : 83093
StellarMasses : 114685
EnvMeasures : 67996
visualMorph : 38795


Join GroupGal & EnvMeasures 24217
GroupGal & EnvMeasures
['CATAID', 'RA_1', 'Dec', 'Z', 'Rpetro', 'DM_100_25_75', 'SURVEY_CODE', 'SigErr', 'GroupID', 'RankIterCen', 'SepIterCen', 'CoSepIterCen', 'AngSepIterCen', 'RankBCG', 'SepBCG', 'CoSepBCG', 'AngSepBCG', 'RankCen', 'SepCen', 'CoSepCen', 'AngSepCen', 'RA_2', 'DEC', 'Z_TONRY', 'nQ', 'M_r', 'DistanceToEdge', 'Completeness', 'DistanceTo5nn', 'SurfaceDensity', 'SurfaceDensityErr', 'SurfaceDensityFlag', 'CountInCyl', 'CountInCylErr', 'CountInCylFlag', 'AGEDenPar', 'AGEErr', 'AGEScale', 'AGEDenParFlag']
24217
Index(['GroupID', 'CountInCyl', 'DistanceTo5nn', 'SurfaceDensity', 'AGEDenPar',
       'Counts', 'Nfof', 'IterCenCATAID', 'IterCenRA', 'IterCenDec',
       'IterCenZ', 'Zfof', 'Rad50', 'Rad1Sig', 'Rad100', 'RelDen', 'Zcomp',
       'GroupEdge', 'VelDisp', 'VelDispRaw', 'VelErr', 'TotRmag',
       'TotFluxProxy', 'MassProxy', 'BCGCATAID', 'BCGRA', 'BCGDec', 'Rgap',
       'CenRA', 'CenDec', 'RadKurt', 'AxAng', 'AxRat', 'SymKS', 'VelSke